Tensor Operations
=======

Naive vs Vectorised
------

Most deep learning algorithms can be reduced to a set of tensor operations.

The FC layer we used earlier in the course:

`keras.layers.Dense(512, activation='relu')`

Can be viewed as a function, the output of which is:

`output = relu(dot(W, input) + b)`

Where W and b are our learned values.

Both relu and addition are element wise operations.
We can implement these operations naively with for loops.

In [7]:
import numpy as np

def naive_relu(x):
    assert len(x.shape) == 2
    
    x = x.copy() #avoids overwriting the original tensor
    
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] = max(x[i, j], 0)
    return x

def naive_add(x, y):
    assert len(x.shape) == 2
    assert x.shape == y.shape
    
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] = x[i, j] + y[i, j]
    return x


x = np.ones((1000,10))
y = np.ones((1000,10))

%timeit naive_relu(x)
%timeit naive_add(x, y)

4.23 ms ± 25.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
3.37 ms ± 26.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


However this tends to be slow. Luckily these elementwise operations are amenable to vectorized implementations. Under the hood numpy uses library such as BLAS to perform these operations at super quick speeds. Rather than writing the for loops explicitly we can instead write them like so.

In [8]:
def vec_relu(x):
    return np.maximum(x, 0)

def vec_add(x, y):
    return x + y

%timeit vec_relu(x)
%timeit vec_add(x, y)

8.34 µs ± 48.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
3.65 µs ± 16.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Broadcasting
------

Our naive implementation supports the addition of 2 tensors of the same shape. But what happens in the vectorised example if the shapes of the tensors differ?

When possible, if no ambiguity exists the smaller tensor will be *broadcasted* into the larger tensor. First axes are added to the smaller tensor so it matches the size of the bigger. Then the smaller tensor is repeated along these new axes.

In [10]:
x = np.ones((12, 10))
y = np.random.rand(10)

print(x+y)

[[1.27555479 1.3224021  1.27638818 1.43912916 1.25725977 1.69828085
  1.73929516 1.10518811 1.04255149 1.69539249]
 [1.27555479 1.3224021  1.27638818 1.43912916 1.25725977 1.69828085
  1.73929516 1.10518811 1.04255149 1.69539249]
 [1.27555479 1.3224021  1.27638818 1.43912916 1.25725977 1.69828085
  1.73929516 1.10518811 1.04255149 1.69539249]
 [1.27555479 1.3224021  1.27638818 1.43912916 1.25725977 1.69828085
  1.73929516 1.10518811 1.04255149 1.69539249]
 [1.27555479 1.3224021  1.27638818 1.43912916 1.25725977 1.69828085
  1.73929516 1.10518811 1.04255149 1.69539249]
 [1.27555479 1.3224021  1.27638818 1.43912916 1.25725977 1.69828085
  1.73929516 1.10518811 1.04255149 1.69539249]
 [1.27555479 1.3224021  1.27638818 1.43912916 1.25725977 1.69828085
  1.73929516 1.10518811 1.04255149 1.69539249]
 [1.27555479 1.3224021  1.27638818 1.43912916 1.25725977 1.69828085
  1.73929516 1.10518811 1.04255149 1.69539249]
 [1.27555479 1.3224021  1.27638818 1.43912916 1.25725977 1.69828085
  1.73929516

As you can tell the smaller tensor of shape (10) is first increased to shape (1, 10). It is then repeated 12 times along this new axis so it is of the shape (12, 10) before the addition occurs.

In terms of efficiency this new tensor is never explicitly created as that would be inneficient. It occurs only at the algorithmic level.

Tensor Dot
-----

The dot product or tensor product is another common operation `x.y`. DO NOT CONFUSE IT WITH THE ELEMENT WISE PRODUCT `x*y`!!

In [14]:
x = np.random.rand(10, 32)
y = np.random.rand(32, 10)

print(np.dot(x, y))

[[ 7.84750744  8.38681162  8.85569691  7.64911099  5.50945767  6.94383679
   6.44136261  6.26346693  6.16513987  7.44493172]
 [ 7.11336813  6.76593849  7.85222978  6.97592458  5.27223657  6.77781766
   5.33321827  6.53015894  4.95191001  6.12925617]
 [10.34004842  9.09215371  9.18170543  8.70208187  7.10557054  8.54436447
   8.47307778  8.30695393  6.79804201  9.74560873]
 [ 9.6571777   9.41904653 10.4058656  10.09287886  7.0616049   8.64824469
   8.35083146  9.28013298  6.15444776 10.25322142]
 [ 6.82626766  7.01434511  8.08307291  6.88862745  5.28847955  6.09679284
   6.59041278  6.12721618  5.15854444  6.9232241 ]
 [ 7.23440819  7.48213174  9.01365618  7.84881519  6.34782022  7.85440641
   6.43102044  6.4522272   5.58419033  7.93090153]
 [ 8.62089136  8.45463802  9.33274931  8.02400947  6.27913525  7.49713346
   6.29127201  7.85910015  5.49322854  7.86559291]
 [ 8.9427716   8.08770469  8.30988043  8.64238506  5.98214055  7.76750047
   8.07846186  7.50257905  6.69012934  8.37185537]


But what is this doing exactly? Lets start with a naive vector implementation.

In [15]:
def naive_vector_dot(x, y):
    assert len(x.shape) == 1
    assert len(y.shape) == 1
    assert x.shape[0] == y.shape[0]
    
    z = 0
    for i in range(x.shape[0]):
        z += x[i] * y[i]
    return z

As you can see here for a vector the dot product is the sum of the element wise product which is a scalar.
Between a matrix x and a vector y it's more complex. The dot product of `x.y` becomes the dot product between y and the rows of x. This can be implemented naively as:

In [28]:
def naive_matrix_vector_dot(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 1
    assert x.shape[1] == y.shape[0]
    
    z = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        for j in range(y.shape[0]):
            z[i] += x[i, j] + y[j]
    return z

In [29]:
def naive_matrix_vector_dot(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 1
    assert x.shape[1] == y.shape[0]
    
    z = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        z[i] = naive_vector_dot(x[i], y)
    return z

*Note* now that ndim>1 this is no longer symmetric. `x.y` is not the same as `y.x`.

And finally a matrix dot product between x and y produces a matrix which is the dot product of all rows of x with all columns of y. Therefore the numbers of columns in x must equal the rows in y.

In [36]:
def naive_matrix_dot(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 2
    assert x.shape[1] == y.shape[0]
    
    z = np.zeros((x.shape[0], y.shape[1]))
    for i in range(x.shape[0]):
        for j in range(y.shape[1]):
            z[i, j] = naive_vector_dot(x[i], y[:, j])
    return z

Or again by re-using our old code.

In [41]:
def naive_matrix_dot(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 2
    assert x.shape[1] == y.shape[0]
    
    z = np.zeros((x.shape[0], y.shape[1]))
    for i in range(x.shape[0]):
        z[i] = naive_matrix_vector_dot(y.T, x[i])
    return z

print(np.isclose(naive_matrix_dot(x, y), np.dot(x, y)))

[[ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True]]


Reference the book for a nice diagram of this interaction. if x is of shape (a, b) and y of (b, c) then the result will be (a, c).

This approach genralises to higher dimensions, for example.
(a, b, c, d) . (d,) -> (a, b, c)
(a, b, c, d) . (d, e) -> (a, b, c, e)

Tensor Reshaping
-------

Another essential operation used throughout deep learning is tensor reshaping. This was used as part of our pre-processing of the MNIST dataset.
`train_images = train_images.reshape((train_images.shape[0], train_images.shape[1]*train_images.shape[2]))`
This operation is easiest to explain with a simple example.

In [44]:
x = np.array([0, 1, 2, 3, 4, 5])
print(x)
print(x.shape)
x = x.reshape((6, 1))
print(x)
print(x.shape)
x = x.reshape((3, 2))
print(x)
print(x.shape)
x = x.reshape((2, 3))
print(x)
print(x.shape)

[0 1 2 3 4 5]
(6,)
[[0]
 [1]
 [2]
 [3]
 [4]
 [5]]
(6, 1)
[[0 1]
 [2 3]
 [4 5]]
(3, 2)
[[0 1 2]
 [3 4 5]]
(2, 3)


There is also a special case of reshape known as the *transpose*. This operations swaps the rows and columns of a matrix so that x[